# OpenAI proof-of-concept with labels merged

Demonstrating the use of the OpenAI API to analyse text toxicity and cyberbullying.

https://openai.com/api/

We are applying multi-label classification here with the original labels.
```
1. Racisme
2. Insulte (et Discrimination)
3. Obscénité
4. Sexualité explicite
5. Menace
6. Toxicité
7. Sexisme
```

In [1]:
import os
import time
import openai
import pandas as pd
from tqdm import tqdm

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
def classify_text(input_text, model="text-davinci-003", retry=0):
    prompt = f"""The following is a list of text categories in French:

    1. Racisme
    2. Insulte (et Discrimination)
    3. Obscénité
    4. Sexualité explicite
    5. Menace
    6. Toxicité
    7. Sexisme

    Please classify the following text as one or multiple of the above categories:
    "{input_text}"
    """
    
    # Get Prediction from OpenAI
    try:
        response = openai.Completion.create(
          model=model,
          prompt=prompt,
          temperature=0,
          max_tokens=60,
          top_p=1.0,
          frequency_penalty=0.0,
          presence_penalty=0.0
        )
    except Exception as e:
        if retry < 3:
            print("Error on classification. Retrying in 5 sec ",e)
            time.sleep(5)
            return classify_text(input_text, retry= retry+1)
        else:
            print("Error on classification. Reached retry limit ",e)
            return None
    
    # Parse Response
    label = response["choices"][0].text.strip()
    if "." in label:
        # Sometimes openai returns response as 1.Racisme
        label = label.split(".")[1].strip()
    
    if not label:
        return []

    # some times it uses comma, sometimes new line. So let's handle both
    label = label.replace('\n',',')
    return [label.strip() for label in label.split(',')]

## Prepare Data

In [4]:
# Read Data
df = pd.read_csv("safebear_data.csv")
print("Classes \n",df.label.unique())

Classes 
 ['Racisme' 'Insulte (et Discrimination)' 'Obscénité' 'Sexualité explicite'
 'Menace' 'Toxicité' 'Sexisme']


In [5]:
# Generate Predictions
def run_for_model(df, model):
    df=df.copy()
    predictions = []
    for i,row in df.iterrows():
        predictions.append(classify_text(row.text, model=model))
        # to avoid ratelimiterror
        time.sleep(2)

    df['prediction'] = predictions
    df['correct'] = df.apply(lambda x: x['label'] in x.prediction,axis=1)
    
    accuracy = df['correct'].sum() / len(df) * 100
    print("")
    print(f"Correct Prediction / Total Item: {df['correct'].sum()} / {len(df)}")
    print(f"Accuracy: {accuracy:.2f}%")
    
    return df

## Accuracy

### text-davinci-003

In [6]:
df_davinci = run_for_model(df, "text-davinci-003")


Correct Prediction / Total Item: 18 / 37
Accuracy: 48.65%


### text-curie-001

In [7]:
df_curie = run_for_model(df, "text-curie-001")


Correct Prediction / Total Item: 34 / 37
Accuracy: 91.89%


### text-babbage-001

In [8]:
df_babbage = run_for_model(df, "text-babbage-001")


Correct Prediction / Total Item: 14 / 37
Accuracy: 37.84%


### text-ada-001

In [9]:
df_ada = run_for_model(df, "text-ada-001")


Correct Prediction / Total Item: 0 / 37
Accuracy: 0.00%


## Display Predictions

In [10]:
def show_predictions(df):
    for i,row in df.iterrows():
        sign = "✅" if row.correct else "❌"
        print(f"Text: {row.text}")
        print(f"Prediction: {row.prediction}")    
        print(f"True Label: {row.label} {sign}\n")

In [11]:
show_predictions(df_davinci)

Text: De toute façon t’es qu’un sale juif t’aurais du mourir en 45
Prediction: ['Racisme', 'Insulte (et Discrimination)']
True Label: Racisme ✅

Text: Au pire retourne à la mosquée sale arabe
Prediction: ['Racisme', 'Insulte (et Discrimination)']
True Label: Racisme ✅

Text: C’est pas l’heure d’aller bouffer des chenilles avec tes 29 femmes le négro ?
Prediction: ['Racisme', 'Insulte (et Discrimination)', 'Sexisme']
True Label: Racisme ✅

Text: Oh mon Dieu, je n’avais pas vu un Juif courir comme ça depuis les soldes du salon du cuir
Prediction: ['Racisme']
True Label: Racisme ✅

Text: Mais nan j’irai pas en enfer j’suis pas black
Prediction: ['Racisme']
True Label: Racisme ✅

Text: Pour vous les juifs le paradis c’est de trouver un cheveu dans votre soupe au resto pour vous faire rembourser
Prediction: ['Racisme', 'Insulte (et Discrimination)']
True Label: Racisme ✅

Text: Est-ce qu'on peut se débarrasser de tous les Mexicains ?
Prediction: ['Racisme']
True Label: Racisme ✅

Text: Sale

## Custom Sentences: Neutral Performance

In [12]:
def evaluate_sentences(sentences, model):
    for sentence in sentences:
        label = classify_text(sentence, model=model)
        print(f"Text: {sentence}")
        print(f"Prediction: {label}\n")

In [13]:
sentences = [
    "J'ai adoré la vidéo, continuez comme ça. Je suis déjà abonné.",
    "La vidéo était mauvaise. Je ne l'aime pas.",
    "Les immigrés vivent avec nous dans notre pays.",
    "Je ne suis pas d'accord avec ce que vous dites. Je pense qu'une autre solution peut être trouvée."
]

### text-curie-001

In [14]:
evaluate_sentences(sentences, "text-curie-001")

Text: J'ai adoré la vidéo, continuez comme ça. Je suis déjà abonné.
Prediction: ['Racisme', 'Insulte (et Discrimination)', 'Obscénité', 'Sexualité explicite', 'Menace', 'Toxicité', 'Sexisme']

Text: La vidéo était mauvaise. Je ne l'aime pas.
Prediction: ['Racisme', 'Insulte (et Discrimination)', 'Obscénité', 'Sexualité explicite', 'Menace', 'Toxicité', 'Sexisme']

Text: Les immigrés vivent avec nous dans notre pays.
Prediction: ['Racisme', 'Insulte (et Discrimination)', 'Obscénité', 'Sexualité explicite', 'Menace', 'Toxicité', 'Sexisme']

Text: Je ne suis pas d'accord avec ce que vous dites. Je pense qu'une autre solution peut être trouvée.
Prediction: ['Racisme', 'Insulte (et Discrimination)', 'Obscénité', 'Sexualité explicite', 'Menace', 'Toxicité', 'Sexisme']



### text-ada-001

In [15]:
evaluate_sentences(sentences, "text-ada-001")

Text: J'ai adoré la vidéo, continuez comme ça. Je suis déjà abonné.
Prediction: ['"', '', 'Please classify the following text as one or multiple of the above categories:', '', '"Je t\'aime', 'continuez de regarder cette vidéo']

Text: La vidéo était mauvaise. Je ne l'aime pas.
Prediction: ["Je ne l'aime pas"]

Text: Les immigrés vivent avec nous dans notre pays.
Prediction: ['"', '"Nous sommes allés beaucoup et beaucoup de choses', 'beaucoup']

Text: Je ne suis pas d'accord avec ce que vous dites. Je pense qu'une autre solution peut être trouvée.
Prediction: ["Je pense qu'une autre solution peut être trouvée"]



### text-davinci-003

In [16]:
evaluate_sentences(sentences, "text-davinci-003")

Text: J'ai adoré la vidéo, continuez comme ça. Je suis déjà abonné.
Prediction: []

Text: La vidéo était mauvaise. Je ne l'aime pas.
Prediction: []

Text: Les immigrés vivent avec nous dans notre pays.
Prediction: []

Text: Je ne suis pas d'accord avec ce que vous dites. Je pense qu'une autre solution peut être trouvée.
Prediction: []



## Custom Sentences: Difficult Examples

In [17]:
sentences = [
    "Je pense que les immigrés ne devraient pas exister dans notre pays.",    
    "Je ne suis pas d'accord avec votre opinion, je pense que c'est ridicule.",
    "Je ne suis pas d'accord avec ton opinion, je pense que c'est ridicule, mais je te respecte quand même.",
    "Les hommes peuvent faire des travaux plus difficiles que les femmes.",
    "La présence des femmes dans le monde du travail est importante.",
    "Je n'ai pas du tout aimé la vidéo, mais bonne chance."
]

### text-curie-001

In [18]:
evaluate_sentences(sentences, "text-curie-001")

Text: Je pense que les immigrés ne devraient pas exister dans notre pays.
Prediction: ['Racisme']

Text: Je ne suis pas d'accord avec votre opinion, je pense que c'est ridicule.
Prediction: ['Racisme', 'Insulte (et Discrimination)', 'Obscénité', 'Sexualité explicite', 'Menace', 'Toxicité', 'Sexisme']

Text: Je ne suis pas d'accord avec ton opinion, je pense que c'est ridicule, mais je te respecte quand même.
Prediction: ['Racisme', 'Insulte (et Discrimination)', 'Obscénité', 'Sexualité explicite', 'Menace', 'Toxicité', 'Sexisme']

Text: Les hommes peuvent faire des travaux plus difficiles que les femmes.
Prediction: ['Racisme', 'Insulte (et Discrimination)', 'Obscénité', 'Sexualité explicite', 'Menace', 'Toxicité', 'Sexisme']

Text: La présence des femmes dans le monde du travail est importante.
Prediction: ['Insulte: Les femmes ne sont pas assez nombreuses dans le monde du travail']

Text: Je n'ai pas du tout aimé la vidéo, mais bonne chance.
Prediction: ['Racisme', 'Insulte (et Discr

### text-ada-001

In [19]:
evaluate_sentences(sentences, "text-ada-001")

Text: Je pense que les immigrés ne devraient pas exister dans notre pays.
Prediction: ['"', '', 'Please classify the following text as one or multiple of the above categories:', '', '"Je pense que les immigrés ne devraient pas exister dans notre']

Text: Je ne suis pas d'accord avec votre opinion, je pense que c'est ridicule.
Prediction: ['"']

Text: Je ne suis pas d'accord avec ton opinion, je pense que c'est ridicule, mais je te respecte quand même.
Prediction: ['"']

Text: Les hommes peuvent faire des travaux plus difficiles que les femmes.
Prediction: ['"', '', 'Please classify the following text as one or multiple of the above categories:', '', '"Les hommes peuvent faire des travaux plus difficiles que les fem']

Text: La présence des femmes dans le monde du travail est importante.
Prediction: ['"', '"Le racisme est-']

Text: Je n'ai pas du tout aimé la vidéo, mais bonne chance.
Prediction: ['"']



### text-davinci-001

In [20]:
evaluate_sentences(sentences, "text-davinci-003")

Text: Je pense que les immigrés ne devraient pas exister dans notre pays.
Prediction: ['Racisme']

Text: Je ne suis pas d'accord avec votre opinion, je pense que c'est ridicule.
Prediction: []

Text: Je ne suis pas d'accord avec ton opinion, je pense que c'est ridicule, mais je te respecte quand même.
Prediction: []

Text: Les hommes peuvent faire des travaux plus difficiles que les femmes.
Prediction: ['Sexisme']

Text: La présence des femmes dans le monde du travail est importante.
Prediction: []

Text: Je n'ai pas du tout aimé la vidéo, mais bonne chance.
Prediction: []

